<a href="https://colab.research.google.com/github/Anshuman-02905/URBAN_MONITORING_SYSTEM/blob/main/DATA_ACQUISITION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install geemap
!pip install geopandas
!apt-get update
!apt-get install libgdal-dev -y
!apt-get install python-gdal -y
!apt-get install python-numpy python-scipy -y

In [ ]:
import json
import geemap
import os
import glob
import gdal ## fingers crossed!
import os
import ee
import pandas as pd
import numpy as np
import math
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
import folium

In [ ]:
path = ["/content/drive/MyDrive/DATASET1/Sentinel_png/","/content/drive/MyDrive/DATASET1/Mask_png/","/content/drive/MyDrive/DATASET1/Sentinel_tif/",
        "/content/drive/MyDrive/DATASET1/Urban_tif/"]
for i in path:
  if(os.path.exists(i)!=True):
    os.mkdir(i)


  
# Getting the list of directories
for i in path:
  dir = os.listdir(i)  
  # Checking if the list is empty or not
  if len(dir) != 0:
    files = glob.glob(i+"*")
    for f in files:
      pass
      #os.remove(f)

In [ ]:
Sentinel_png="/content/drive/MyDrive/DATASET1/Sentinel_png/"
urban_png="/content/drive/MyDrive/DATASET1/Mask_png/"
Sentinel_tif="/content/drive/MyDrive/DATASET1/Sentinel_tif/"
Mask_tif="/content/drive/MyDrive/DATASET1/Urban_tif/"
filename="/content/drive/MyDrive/DATASET1/ACTIVE_LOG.json"  


In [ ]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=EqmWeWor_Xe-JQBIZrV3ZdvrGY2DtLZcsneWcYTdV1A&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjUB5yKwPyRnQPrb_NOI7fOydapJLwJx-cugvuv0FM0s2awwq2l-nI

Successfully saved authorization token.


In [ ]:
def func(c):
  try:
    mine_shp="/content/newboundary.shp"
    mine = geemap.shp_to_ee(mine_shp) 
    Map=geemap.Map()
    state = mine
    #Map=geemap.Map()
    #state = ee.FeatureCollection("FAO/GAUL/2015/level2").filter(ee.Filter.eq('ADM2_NAME','Bangalore Urban'))
    sentinel = ee.ImageCollection('COPERNICUS/S2_SR').select(['B4', 'B3', 'B2']) \
                  .filterDate('2017-01-01', '2020-01-30') \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5)) \
                  .filterBounds(state)
    image = sentinel.mean().divide(10000).clip(state).select(['B4', 'B3', 'B2']).unmask()

    visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
    }
    #Map.addLayer(image, visualization, 'RGB')

    mask = ee.Image("DLR/WSF/WSF2015/v1").clip(state)

    visualization = {
    'min': 0,
    'max': 255,
    }

    #Map.addLayer(urban.clip(state), visualization, "Human1")
    mask=mask.expression('b("settlement") == 255 ? 255:0',{'settlement':mask.select('settlement')}).clip(state)

    #Map.addLayer(mask, visualization, "Human2")
    #Map.centerObject(image)
    Sentinel=Sentinel_tif+str(c)+".tif"
    urban=Mask_tif+str(c)+".tif"

    geemap.ee_export_image(mask, filename=urban, scale=10, region=state.geometry(), file_per_band=False)
    geemap.ee_export_image(image, filename=Sentinel, scale=10, region=state.geometry(), file_per_band=False)
    Sentinel_mask=Sentinel_png+str(c)+".png"
    Mask_mask=urban_png+str(c)+".png"
    options_list = ['-ot Byte','-of PNG','-scale 0 0.3 0 255'] 
    options_string = " ".join(options_list) 
    gdal.Translate(Sentinel_mask,Sentinel, options=options_string)
    options_list = ['-ot Byte','-of PNG'] 
    options_string = " ".join(options_list)
    gdal.Translate(Mask_mask, urban, options=options_string)
    xml1=Sentinel_mask+".aux.xml"
    xml2=Mask_mask+".aux.xml"
    os.remove(xml1)
    os.remove(xml2)
    os.remove(urban)
    os.remove(Sentinel)
    return 1
  except Exception as e:
    print(e)
    print("Exception")
    return -1

In [ ]:
def get_boundaries(final_list,c):
  coordinates = final_list
  def create_polygon(coords, polygon_name):
    ''' Create a polygon from coordinates'''
    polygon = Polygon(coordinates)
    gdf = gpd.GeoDataFrame(crs = {'init' :'epsg:4326'})
    gdf.loc[0,'name'] = polygon_name
    gdf.loc[0, 'geometry'] = polygon
    return gdf
  shapefile = create_polygon(coordinates, "Amesterdam")
  shapefile.to_file("newboundary.shp")
  mine_shp="/content/newboundary.shp"
  mine = geemap.shp_to_ee(mine_shp)
  return func(c)
  

In [ ]:
def get_final_list(Lon,Lat):
    LL=[Lon,Lat]
    UL=[Lon,Lat+left_dist]
    LR=[Lon+lower_dist,Lat]
    UR=[Lon+lower_dist,Lat+left_dist]
    final_list=[LL,LR,UR,UL,LL]
    return final_list

In [ ]:
fl=[[72.72114537234593, 23.96269296245415], [72.86143323952895, 23.96269296245415], [72.86143323952895, 24.06607193151615], [72.72114537234593, 24.06607193151615], [72.72114537234593, 23.96269296245415]]

In [ ]:
left_dist=0.1033789690619983
right_dist=0.1033789690619983
upper_dist=0.1402878671830234
lower_dist=0.1402878671830234

In [ ]:
def update_error(c,final_list_string,error):
  data=dict()
  f = open(r"/content/drive/MyDrive/DATASET1/LOG.txt", "a")
  f.write("EXCEPTION"+str(c)+final_list_string+"\n")
  f.close()
  error+=1
  with open(filename) as json_file:
   data = json.load(json_file)
   data["Error_num"]=error
  with open(filename, 'w') as file_object:  #open the file in write mode
   json.dump(data, file_object)
   


In [ ]:
final_list_string = ' '.join([str(elem) for elem in fl])


In [ ]:
#update_error(0,final_list_string,2)

In [ ]:
final_list=list()
count=0
#cord=[0, 0]
#cord=[23.96269296245415, 72.72114537234593]
#cord=[25.409998529322145,72.72114537234593]
#cord=[26.133651312756143,72.72114537234593]
cord=[27.68433584868614,72.72114537234593]


data=dict()
with open(filename) as json_file:
    data = json.load(json_file)
Lat=data["Active_latitude"]
i=data["Active_I"]
error=data['Error_num']
c=data["Lat_c"]
Lon=cord[1]

for j in range(i,100):
  Lon=cord[1]
  data=dict()
  with open(filename) as json_file:
      data = json.load(json_file)
  data["Active_latitude"]=Lat
  data["Lat_c"]=c
  data['Active_I']=j
  with open(filename, 'w') as file_object:  #open the file in write mode
   json.dump(data, file_object)

  for j in range(0,100):
    with open(filename) as json_file:
     data = json.load(json_file)
     error=data['Error_num']
    print("Image_number",c)
    final_list=get_final_list(Lon,Lat)
    final_list_string = ' '.join([str(elem) for elem in final_list])
    num=str(c)
    f = open(r"/content/drive/MyDrive/DATASET1/LOG.txt", "a")
    f.write(num+"\n"+final_list_string+"\n")
    f.close()

    result=get_boundaries(final_list,c)

    if result==-1:
     update_error(c,final_list_string,error)

    if result!=-1:
      c+=1

    path = ["/content/drive/MyDrive/DATASET1/Sentinel_tif/","/content/drive/MyDrive/DATASET1/Urban_tif/"]

    # Getting the list of directories
    for i in path:
      dir = os.listdir(i)  
      # Checking if the list is empty or not
      if len(dir) != 0:
        files = glob.glob(i+"*")
        for f in files:
          os.remove(f)

    Lon+=lower_dist
  Lat+=left_dist

Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/DATASET1/Urban_tif/5123.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/DATASET1/Sentinel_tif/5123.tif


In [ ]:
import json
import os
filename="/content/drive/MyDrive/DATASET1/ACTIVE_LOG.json"  
data=dict()
with open(filename) as json_file:
    data = json.load(json_file)
Lat=data["Active_latitude"]
i=data["Active_I"]
error=data['Error_num']

In [ ]:
print(i)

99


In [ ]:
print(c)

9900


In [ ]:
print(23.96269296245415+9*0.1033789690619983)

24.893103684012132
